In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls /content/drive/MyDrive

Mounted at /content/drive
 20191216_122821.mp4		  gptneo-finetuned-qa
 20200101_074036.mp4		  math
 20200324_211649.mp4		 'Screenshot_20220307-174728_().jpg'
 Classroom			  대마도여행250224.gmap
'Colab Notebooks'		  여수여행_241216.gmap
 Colab_Notebooks		 '역사 보고서.show'
'Gantt Chart_07의 사본.gslides'   오키나와여행_241230.gmap
 gptneo-1.3B-finetuned		  통지서.html
 gptneo-1.3B-masked		  홍콩여행_240121.gmap


In [2]:
# 1. 필요한 패키지 설치
!pip install --upgrade transformers datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import json
import torch
from datasets import Dataset
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 모델 및 토크나이저 로딩
model_name = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token

tokenizer.add_special_tokens({'eos_token': '</s>', 'pad_token': '<pad>'})
model.resize_token_embeddings(len(tokenizer))

print("eos_token:", tokenizer.eos_token)
print("pad_token:", tokenizer.pad_token)
print("eos_token_id:", tokenizer.eos_token_id)
print("pad_token_id:", tokenizer.pad_token_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

eos_token: </s>
pad_token: <pad>
eos_token_id: 1
pad_token_id: 3


In [4]:
# 3. 수학문제 데이터
from google.colab import files
import json

uploaded = files.upload()

with open("augmented_math.json", "r", encoding="utf-8") as f:
    math_data = json.load(f)

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

Saving augmented_math.json to augmented_math.json


In [10]:
# 3. 상식문제 데이터
from google.colab import files
import json

uploaded = files.upload()

with open("augmented_knowledge.json", "r", encoding="utf-8") as f:
    knowledge_data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'augmented_knowledge.json'

In [ ]:
# 3. 논리문제 데이터
from google.colab import files
import json

uploaded = files.upload()

with open("augmented_logic.json", "r", encoding="utf-8") as f:
    logic_data = json.load(f)

In [ ]:
# 3. 정답이 없는 문제 데이터
from google.colab import files
import json

uploaded = files.upload()

with open("augmented_open_problem.json", "r", encoding="utf-8") as f:
    open_question_data = json.load(f)

In [5]:
# 4. Math 문제+답 텍스트 생성
formatted_data = []
for item in math_data:
    prompt = f"질문: {item['Question']}\n답: {item['Answer']}"
    formatted_data.append({"Text": prompt})

In [6]:
# 토크나이즈 함수
def tokenize(example):
    return tokenizer(example["Text"], padding="max_length", truncation=True, max_length=512)

In [7]:
# 6. Dataset 변환 및 전처리
from datasets import Dataset

dataset = Dataset.from_list(formatted_data)
tokenized_dataset = dataset.map(tokenize, batched=False)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [18]:
import transformers
import inspect

print(transformers.__file__)  # 실제 불러온 transformers 위치 확인
print(inspect.getfile(transformers.TrainingArguments))  # TrainingArguments가 로드된 경로


/usr/local/lib/python3.11/dist-packages/transformers/__init__.py
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py


In [14]:
# 7. Trainer 준비
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./kogpt2-finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=10,
    eval_strategy="no",
    fp16=torch.cuda.is_available(),
    overwrite_output_dir=True
)

In [16]:
# 데이터 collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
# 8. Trainer 구성 & 학습 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

<ipython-input-19-ceccad0ce5d7>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: circlehalf17 (circlehalf17-no-job) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.982700
20,2.710700
30,3.042300
40,3.159600
50,2.343800
60,2.410400
70,2.177800
80,2.237600
90,1.737100
100,1.988600


TrainOutput(global_step=1000, training_loss=0.9988429238796234, metrics={'train_runtime': 176.2347, 'train_samples_per_second': 11.349, 'train_steps_per_second': 5.674, 'total_flos': 522584064000000.0, 'train_loss': 0.9988429238796234, 'epoch': 5.0})

In [20]:
model.save_pretrained("/content/drive/MyDrive/math")
tokenizer.save_pretrained("/content/drive/MyDrive/math")

('/content/drive/MyDrive/math/tokenizer_config.json',
 '/content/drive/MyDrive/math/special_tokens_map.json',
 '/content/drive/MyDrive/math/tokenizer.json')